In [2]:
# import  libraries
import pandas as pd
import numpy as np
from datetime import timedelta
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import ipywidgets as widgets
from IPython.display import display



In [3]:
# read excel file 

samp_orders = pd.read_excel("sample.xlsx")

In [4]:
# convert sales order date to a datetime datatype 

samp_orders["sal_orderDate"] = pd.to_datetime(samp_orders["sal_orderDate"])

In [5]:
# convert table to a dataframe
samp_orders_df =pd.DataFrame(samp_orders)
# print(samp_orders_df.head(5))

# delete column with index
del samp_orders_df["Unnamed: 0"]
# print(sam_orders_df.head(5))

# check the number of null values in a  column
# print(samp_orders_df.isnull().sum())

# # drop or delete all null rows
# samp_orders_df.dropna(inplace=True)
# # print(sam_orders_df)
# print(samp_orders_df.isnull().sum())

In [6]:
# # Feature Engineering: Encode CustomerID and ProductID
# customer_encoder = LabelEncoder()
# product_encoder = LabelEncoder()

# samp_orders_df['CustomerID_enc'] = customer_encoder.fit_transform(samp_orders_df["sal_custID"])
# samp_orders_df['ProductID_enc'] = product_encoder.fit_transform(samp_orders_df["sal_Prod_id"])

# #Create lag features: Previous ProductID and PurchaseDate
# samp_orders_df['PreviousProductID'] = samp_orders_df.groupby('CustomerID_enc')['ProductID_enc'].shift(1)
# samp_orders_df['DaysSinceLastPurchase'] = samp_orders_df.groupby('CustomerID_enc')['sal_orderDate'].diff().dt.days

# # Drop rows with NaN values resulting from shift operation
# samp_orders_df.dropna(inplace=True)

# print(samp_orders_df)



# new feature engineering for accuracy
# Feature Engineering
samp_orders["sal_orderDate"] = pd.to_datetime(samp_orders["sal_orderDate"])
samp_orders['DayOfWeek'] = samp_orders['sal_orderDate'].dt.dayofweek
samp_orders['Month'] = samp_orders['sal_orderDate'].dt.month
samp_orders['DaysSinceLastOrder'] = samp_orders.groupby('sal_custID')['sal_orderDate'].diff().dt.days

# Aggregation Features
customer_agg = samp_orders.groupby('sal_custID').agg({
    'sal_orderID': 'count',  # Total purchases
    'sal_turnover': 'mean',  # Average purchase value
    'sal_orderqty': 'mean'   # Average order quantity
}).rename(columns={'sal_orderID': 'TotalPurchases', 'sal_turnover': 'AvgTurnover', 'sal_orderqty': 'AvgOrderQty'})

samp_orders = samp_orders.merge(customer_agg, on='sal_custID', how='left')

# Encoding
label_encoders = {}
for col in ['sal_Prod_Grp', 'sal_Prod_id', 'sal_Prod_Name', 'sal_custID']:
    le = LabelEncoder()
    samp_orders[col + '_enc'] = le.fit_transform(samp_orders[col])
    label_encoders[col] = le

# Encoding additional categorical columns
# label_encoders = {}
# for column in ['sal_Prod_Grp', 'sal_Prod_Name']:
#     le = LabelEncoder()
#     samp_orders_df[column + '_enc'] = le.fit_transform(samp_orders_df[column])
#     label_encoders[column] = le


# Fill NaN values (e.g., from DaysSinceLastOrder)
samp_orders['DaysSinceLastOrder'].fillna(0, inplace=True)




/var/folders/cl/_zhh_drj7jsdmzq8d42x8rrr0000gr/T/ipykernel_1299/692735538.py:51: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  samp_orders['DaysSinceLastOrder'].fillna(0, inplace=True)


In [7]:
# # Define feature columns and target
# features = ['CustomerID_enc', 'PreviousProductID', 'DaysSinceLastPurchase']
# target = 'ProductID_enc'
# # Split data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(samp_orders_df[features], samp_orders_df[target], test_size=0.2, random_state=42)

# print(X_train)
# print(X_test)
# print(y_test)
# print(y_train)


# new definition for ffeature columns and targets
# Define features and target
features = [
     'sal_Prod_Grp_enc', 'sal_Prod_id_enc', 'sal_custID_enc', 'DayOfWeek', 'Month',
    'DaysSinceLastOrder', 'TotalPurchases', 'AvgTurnover', 'AvgOrderQty'
]
target = 'sal_Prod_id_enc'

# Split data
X_train, X_test, y_train, y_test = train_test_split(samp_orders[features], samp_orders[target], test_size=0.2, random_state=42)

In [8]:
# unique_classes = set(y_train)

# unique_classes = sorted(list(unique_classes))
# print(f'Unique classes in y_train: {unique_classes}')
# print(f'Number of unique classes: {len(unique_classes)}')

# product_encoder = LabelEncoder()
# product_encoder.fit(unique_classes)

# y_train_encoded = product_encoder.transform(y_train)

# # num_classes = len(set(y_train_encoded))
# num_classes = len(product_encoder.classes_)  # Or len(set(y_train_encoded))
# model = xgb.XGBClassifier(objective='multi:softprob', num_class=num_classes, eval_metric='mlogloss')



# # Define the model
# model = xgb.XGBClassifier(objective='multi:softprob', num_class=len(product_encoder.classes_), eval_metric='mlogloss')

# # Train the model
# model.fit(X_train, y_train)

# # Make predictions
# y_pred = model.predict(X_test)
# print(samp_orders_df[target])

# # Check and print unique classes in y_train
# unique_classes = sorted(list(set(y_train)))
# print(f'Unique classes in y_train: {unique_classes}')
# print(f'Number of unique classes: {len(unique_classes)}')

# # Initialize and fit the LabelEncoder
# product_encoder = LabelEncoder()
# product_encoder.fit(unique_classes)

# # Transform y_train
# y_train_encoded = product_encoder.transform(y_train)
# print(f'Encoded y_train: {y_train_encoded}')
# print(f'Unique classes in encoded y_train: {set(y_train_encoded)}')

# # Define the model with the correct number of classes
# num_classes = len(set(y_train_encoded))
# model = xgb.XGBClassifier(objective='multi:softprob', num_class=num_classes, eval_metric='mlogloss')

# # Train the model
# model.fit(X_train, y_train_encoded)

# # Initialize the model
# model = RandomForestClassifier(random_state=42)

# # Train the model
# model.fit(X_train, y_train_encoded)


# new training for random forest model
# Train a Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
rf_model.fit(X_train, y_train)


RandomForestClassifier(class_weight='balanced', random_state=42)

In [9]:
# # Make predictions
# y_pred = model.predict(X_test)

# print(y_pred)

# new predictions and evaluations
# Predictions and evaluation
y_pred_rf = rf_model.predict(X_test)
print(f'Random Forest Accuracy: {accuracy_score(y_test, y_pred_rf):.2f}')
print(classification_report(y_test, y_pred_rf, digits=3))

Random Forest Accuracy: 0.99
              precision    recall  f1-score   support

           1      0.000     0.000     0.000         3
           2      0.972     1.000     0.986       141
           3      1.000     1.000     1.000       219
           4      1.000     1.000     1.000         4
           5      1.000     1.000     1.000       285
           6      1.000     1.000     1.000        59
           7      1.000     1.000     1.000         2
           8      1.000     1.000     1.000        13
           9      1.000     1.000     1.000        18
          10      1.000     1.000     1.000       168
          11      0.000     0.000     0.000         2
          12      0.913     0.988     0.949        85
          13      0.000     0.000     0.000         6
          14      0.000     0.000     0.000         1
          15      0.991     1.000     0.995       218
          16      1.000     1.000     1.000        60
          17      1.000     0.996     0.998       27

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

In [10]:
# new_data = samp_orders_df[['CustomerID_enc', 'PreviousProductID', 'DaysSinceLastPurchase']]
# predictions = model.predict_proba(new_data)
# print(predictions[0:5])

In [11]:
# predictions_target = predictions

# # print(type(target))
# target = np.array(target)  # Convert to a NumPy array if it's not already


# if target.ndim == 0:
#     print(f"Target: {target}")
# else:
#     print(f"Target array: {target[:5]}, dtype: {target.dtype}, shape: {target.shape}")

# print(f"Target array: {target[:5]}, dtype: {target.dtype}, shape: {target.shape}")
# print(f"Predictions array: {predictions_target[:5]}, dtype: {predictions_target.dtype}, shape: {predictions_target.shape}")


# # Calculate the AUC value
# auc = roc_auc_score(samp_orders_df[target], predictions_target, multi_class='ovr')
# print(round(auc,2))

In [13]:
# # Convert probabilities to class labels
# if y_pred.ndim > 1:
#     y_pred = y_pred.argmax(axis=1)

# print(f'y_test shape: {y_test.shape}, y_test type: {y_test.dtype}')
# print(f'y_pred shape: {y_pred.shape}, y_pred type: {y_pred.dtype}')

# # # Make predictions and convert to class labels
# y_pred = model.predict(X_test)

# # Ensure y_test and y_pred are of the same shape and type
# y_test = y_test.astype(int)  # Ensuring y_test is integer type
# y_pred = y_pred.astype(int)  # Ensuring y_pred is integer type

# # Calculate accuracy
# accuracy = accuracy_score(y_test, y_pred)
# print(f'Accuracy: {accuracy:.2f}')

# # Predict on the test set
# print(classification_report(y_test, y_pred, digits=3))

In [14]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)
print(f'Random Forest Accuracy: {accuracy_score(y_test, y_pred_rf):.2f}')
print(classification_report(y_test, y_pred_rf, digits=3))


Random Forest Accuracy: 0.99
              precision    recall  f1-score   support

           1      0.000     0.000     0.000         3
           2      0.972     1.000     0.986       141
           3      1.000     1.000     1.000       219
           4      1.000     0.500     0.667         4
           5      0.993     1.000     0.997       285
           6      1.000     1.000     1.000        59
           7      0.500     0.500     0.500         2
           8      1.000     0.692     0.818        13
           9      0.857     1.000     0.923        18
          10      1.000     1.000     1.000       168
          11      1.000     0.500     0.667         2
          12      0.912     0.976     0.943        85
          13      0.000     0.000     0.000         6
          14      0.000     0.000     0.000         1
          15      0.986     0.995     0.991       218
          16      1.000     1.000     1.000        60
          17      1.000     1.000     1.000       27

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_pr

In [15]:
# from sklearn.linear_model import LogisticRegression
# lr_model = LogisticRegression(multi_class='ovr', max_iter=1000)
# lr_model.fit(X_train, y_train)
# y_pred_lr = lr_model.predict(X_test)
# print(f'Logistic Regression Accuracy: {accuracy_score(y_test, y_pred_lr):.2f}')
# print(classification_report(y_test, y_pred_lr, digits=3))


In [16]:
# from sklearn.ensemble import GradientBoostingClassifier
# gbm_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
# gbm_model.fit(X_train, y_train)
# y_pred_gbm = gbm_model.predict(X_test)
# print(f'GBM Accuracy: {accuracy_score(y_test, y_pred_gbm):.2f}')
# print(classification_report(y_test, y_pred_gbm, digits=3))


In [17]:
# from sklearn.svm import SVC
# svm_model = SVC(kernel='linear', probability=True)
# svm_model.fit(X_train, y_train)
# y_pred_svm = svm_model.predict(X_test)
# print(f'SVM Accuracy: {accuracy_score(y_test, y_pred_svm):.2f}')
# print(classification_report(y_test, y_pred_svm, digits=3))


In [18]:
# from sklearn.neighbors import KNeighborsClassifier
# knn_model = KNeighborsClassifier(n_neighbors=5)
# knn_model.fit(X_train, y_train)
# y_pred_knn = knn_model.predict(X_test)
# print(f'KNN Accuracy: {accuracy_score(y_test, y_pred_knn):.2f}')
# print(classification_report(y_test, y_pred_knn, digits=3))


In [19]:
import lightgbm as lgb
lgb_model = lgb.LGBMClassifier(n_estimators=100, random_state=42)
lgb_model.fit(X_train, y_train)
y_pred_lgb = lgb_model.predict(X_test)
print(f'LightGBM Accuracy: {accuracy_score(y_test, y_pred_lgb):.2f}')
print(classification_report(y_test, y_pred_lgb, digits=3))


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000461 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 662
[LightGBM] [Info] Number of data points in the train set: 37469, number of used features: 9
[LightGBM] [Info] Start training from score -8.739510
[LightGBM] [Info] Start training from score -9.432657
[LightGBM] [Info] Start training from score -4.264069
[LightGBM] [Info] Start training from score -3.642697
[LightGBM] [Info] Start training from score -8.228684
[LightGBM] [Info] Start training from score -3.611585
[LightGBM] [Info] Start training from score -5.021881
[LightGBM] [Info] Start training from score -8.585359
[LightGBM] [Info] Start training from score -6.920351
[LightGBM] [Info] Start training from score -6.187464
[LightGBM] [Info] Start training from score -4.119451
[LightGBM] [Info] Start training from score -8.739510
[

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

In [20]:
# Example: Create new customer data (replace with actual data)
# Assume CustomerID 1, previous product ID 103, and 30 days since the last purchase
# new_customer_data = pd.DataFrame({
#     'CustomerID_enc': [customer_encoder.transform([1])[0]],  # Example CustomerID
#     'PreviousProductID': [product_encoder.transform([103])[0]],  # Previous ProductID
#     'DaysSinceLastPurchase': [30]  # Days since last purchase
# })
# # Assuming your original DataFrame is named 'original_df' and has columns 'ProductID' and 'ProductName'
# product_mapping = samp_orders_df.set_index("sal_Prod_id_enc")["sal_Prod_Name"].to_dict()


# model_df = samp_orders_df[['CustomerID_enc', 'PreviousProductID', 'DaysSinceLastPurchase']]
# # Predict and convert to class label if necessary
# next_product_encoded = rf_model.predict(model_df)

# # Check if next_product_encoded is a 2D array (indicating probabilities)
# if next_product_encoded.ndim > 1:
#     next_product_encoded = next_product_encoded.argmax(axis=1)

# # Decode the product name
# next_product_name = le.inverse_transform(next_product_encoded)[0]
# print(f'The next product the customer is likely to buy is: {next_product_name}')


# Step 1: Create the product mapping (ProductID_enc to Product Name)
product_mapping = samp_orders.set_index("sal_Prod_id_enc")["sal_Prod_Name"].to_dict()

# Step 2: Predict the next product for each row in the test set
model_df = X_test

# Step 3: Predict the next product
next_product_encoded = rf_model.predict(model_df)

# Step 4: Decode the predicted product ID to the product name
next_product_name = [product_mapping[encoded_id] for encoded_id in next_product_encoded]

# Optionally, you can print the first few predictions to see the results
for i in range(5):
    print(f"Predicted encoded product ID: {next_product_encoded[i]}, Product Name: {next_product_name[i]}")


    



Predicted encoded product ID: 43, Product Name: nan
Predicted encoded product ID: 73, Product Name: Onga Shrimp 24 x 64 x12gr tablet
Predicted encoded product ID: 37, Product Name: Kremela Plain 210 x 20 gr sachet
Predicted encoded product ID: 57, Product Name: Onga Chicken 200x10gr sachet
Predicted encoded product ID: 95, Product Name: Yumvita Wheat 80x50gr sachet


In [30]:
# # Predict and convert to class label if necessary
# next_product_encoded = rf_model.predict(model_df)

# # Check if next_product_encoded is a 2D array (indicating probabilities)
# if next_product_encoded.ndim > 1:
#     next_product_encoded = next_product_encoded.argmax(axis=1)

# # Decode the predicted product ID
# next_product_id = le.inverse_transform(next_product_encoded)[0]
# next_product_name = product_mapping[next_product_id]  # Assuming you have a dictionary mapping product IDs to names

# # Decode the customer ID
# customer_id_enc = model_df['CustomerID_enc'].iloc[0]
# customer_id = le.inverse_transform([customer_id_enc])[0]

# print(f'The next product the customer {customer_id} is likely to buy is: {next_product_name}')


# new code
# # Step 1: Create the product mapping (ProductID_enc to Product Name)
# product_mapping = samp_orders.set_index("sal_Prod_id_enc")["sal_Prod_Name"].to_dict()

# # Step 2: Predict the next product for each customer in the test set
# model_df = X_test

# # Step 3: Predict the next product (encoded)
# next_product_encoded = rf_model.predict(model_df)

# # Step 4: Check if the predictions are a 2D array (indicating probabilities) and convert to class labels if necessary
# if next_product_encoded.ndim > 1:
#     next_product_encoded = next_product_encoded.argmax(axis=1)

# # Step 5: Loop through the first 5 customers and print the predicted product
# for i in range(5):
#     # Decode the predicted product ID to the original product name
#     predicted_product_name = product_mapping[next_product_encoded[i]]
    
#     # Decode the customer ID
#     customer_id_enc = model_df['sal_custID_enc'].iloc[i]
#     customer_id = label_encoders['sal_custID'].inverse_transform([customer_id_enc])[0]
    
#     # Print the result for each customer
#     print(f'Customer {customer_id} is likely to buy: {predicted_product_name}')
# Filter out any NaN values in customer IDs
X_test = X_test.dropna(subset=['sal_custID_enc'])

# Get unique encoded customer IDs
unique_customer_ids = X_test['sal_custID_enc'].unique()

# Function to display prediction for a selected customer
def display_prediction(customer_id):
    customer_data = X_test[X_test['sal_custID_enc'] == customer_id]
    
    if customer_data.empty:
        print(f"No test data available for Customer {customer_id}.")
        return
    
    predicted_product_encoded = rf_model.predict(customer_data)
    predicted_product_name = product_mapping[predicted_product_encoded[0]]
    
    decoded_customer_id = label_encoders['sal_custID'].inverse_transform([customer_id])[0]
    print(f'The next product Customer {decoded_customer_id} is likely to buy is: {predicted_product_name}')

# Dropdown menu for selecting customer
customer_dropdown = widgets.Dropdown(
    options=[(str(label_encoders['sal_custID'].inverse_transform([id_])[0]), id_) for id_ in unique_customer_ids],
    description='Customer:',
    disabled=False,
)

# Update function to call when dropdown value changes
def on_change(change):
    if change['new']:
        encoded_customer_id = change['new']
        display_prediction(encoded_customer_id)

customer_dropdown.observe(on_change, names='value')

# Display the dropdown
display(customer_dropdown)

Dropdown(description='Customer:', options=(('nan', np.int64(129)), ('CL0079', np.int64(44)), ('CF0014', np.int…

The next product Customer CL0079 is likely to buy is: Kremela Plain 210 x 20 gr sachet
The next product Customer CF0014 is likely to buy is: Onga Chicken 200x10gr sachet
The next product Customer CL0192 is likely to buy is: Cowbell Strawberry 100x40gr sachet
The next product Customer CL0314 is likely to buy is: Cowbell Strawberry 100x40gr sachet
The next product Customer CL0372 is likely to buy is: Cowbell Banana 100x40gr sachet
The next product Customer CL0129 is likely to buy is: Yumvita Wheat 80x50gr sachet
The next product Customer nan is likely to buy is: nan
The next product Customer CL0437 is likely to buy is: Yumvita Maize/Wheat 12 x 400gr tin
The next product Customer CL0158 is likely to buy is: Yumvita Maize/Wheat 12 x 400gr tin
The next product Customer CL0337 is likely to buy is: Cowbell Banana 100x40gr sachet
